In [16]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyhecdss
from vtools.data.vtime import hours
from vtools.functions.filter import cosine_lanczos
import altair as alt
from tabulate import tabulate

pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)     # Show all rows
pd.set_option('display.width', 1000)        # Adjust the width to fit the DataFrame
pd.set_option('display.colheader_justify', 'left')  # Left-align column headers
pd.set_option('display.float_format', '{:.2f}'.format)

In [2]:
def load_dss(dss_file,run_list, C_part):
    '''
    Load dss files and output dataframe
    Parameters
    ----------
    dss_file : str
        filename and path of the dss file to load.
    run_list : LIST
        List of the station names wanted in the dataframe.
    C_part : str
        Name of the part C wanted, i.e. 'STAGE' or 'FLOW'.

    Returns
    -------
    temp : dataframe
        dataframe containing the time serie for the stations specified..

    '''
    
    temp = pd.DataFrame()
    # read dss catalog
    with pyhecdss.DSSFile(dss_file) as d:
         df=d.read_catalog()
    sites = df['B'].where(df['C']==C_part).dropna()
    common_sites = list(set(run_list).intersection(set(sites)))
    catdf = df[df['B'].isin(common_sites)] # filter by needed observation stations
    catdf.reset_index(drop=True, inplace=True)

    if C_part == 'STAGE':
        cat = catdf[catdf['C']=='STAGE']
        plists=d.get_pathnames(cat) # that's the list of ts paths
        for i, plist in enumerate(plists):  
            temp[i],units1,ptype1=d.read_rts(plist) # couldn't figure out how to do it without iterating 
            #print(i, plist)

    if C_part == 'FLOW':
        cat = catdf[catdf['C']=='FLOW']
        plists=d.get_pathnames(cat) # that's the list of ts paths
        for i, plist in enumerate(plists):  
            temp[i],units1,ptype1=d.read_rts(plist)
            
    if C_part == 'EC':
        cat = catdf[catdf['C']=='EC']
        plists=d.get_pathnames(cat) # that's the list of ts paths
        for i, plist in enumerate(plists):  
            temp[i],units1,ptype1=d.read_rts(plist) 
        #print(i, plist)
        
    if C_part == 'DEVICE-FLOW':
        cat = catdf[catdf['C']=='DEVICE-FLOW']
        plists=d.get_pathnames(cat) # that's the list of ts paths
        for i, plist in enumerate(plists):  
            temp[i],units1,ptype1=d.read_rts(plist) 
            #print(i, plist)
            
    if C_part == 'ELEV':
        cat = catdf[catdf['C']=='ELEV']
        plists=d.get_pathnames(cat) # that's the list of ts paths
        for i, plist in enumerate(plists):  
            temp[i],units1,ptype1=d.read_rts(plist) 
            #print(i, plist)    
    #A = cat.B.str.replace('GLC2','GLC',regex=True)
    #B = A.str.replace('VCU2','VCU',regex=True)
    temp.columns = cat.B
    
    #temp = temp[common_sites]

    print(dss_file, C_part)
    temp.info()
    return temp


In [3]:
#%% Load data
# leg1 = 'Modeled Historical (H2M)'
leg1 = 'Modeled Historical (H1Z)'
scenario = 'FPV2Ma'

# For now I'll change it by hand but later will read it from echo file
gatef = {'name' : ['GrantLine','MiddleRiver','OldRiver'],
         'width' : [5,5,5],
         'C' : [0.8,0.8,0.8],
         'bottom_elev' :[-6,-5,-7],
         'ID': ['GLC','MID','OLD'],
         'station' : ['DGL','MHO','OLD']}

fig_folder = r'postpro\fig'
fig_name = scenario+'_compv1_%s.png'
save_fig = True
year = '2021'
leg2 = scenario

stime = '%s-01-01'%year
etime = '%s-12-31'%year

# Scenario Developped
hydro2 = r'.\studies\{}\output\{}_hydro.dss'.format(scenario, scenario)
SDG2 = r'.\studies\{}\output\{}_SDG.dss'.format(scenario, scenario)


# Baseline run for hydro comparison
# hydro1 = r'.\studies\H2M\output\H2M_hydro.dss'
#gtm1 = r'../plan_studies/PRIO_og/output/PRIO_og_gtm.dss'
# SDG1 = r'..\studies\H2M\output\H2M_SDG.dss'

hydro1 = r'.\studies\H2M\output\H1Z_0_hydro.dss'
SDG1 = r'..\studies\H2M\output\H1Z_0_CCF.dss'



In [4]:
# load data for Velocity analysis
C_part = 'STAGE' # 
elev_list =['MID_GATE_UP','MID_GATE_DOWN','GLC_GATE_UP','GLC_GATE_DOWN','OLD_GATE_UP','OLD_GATE_DOWN']
flow_list = ['GLC_FLOW_FISH','MID_FLOW_FISH','MID_FLOW_GATE','OLD_FLOW_FISH','OLD_FLOW_GATE']

#sc1_stage = load_dss(SDG1,elev_list, C_part)
sc2_stage = load_dss(SDG2,elev_list, C_part)
sc2_stage = sc2_stage[stime:etime]

C_part = 'DEVICE-FLOW'
#sc1_flow= load_dss(SDG1,flow_list, C_part)
sc2_flow = load_dss(SDG2,flow_list, C_part)
sc2_flow = sc2_flow[stime:etime]

# Load data for WL compliance
C_part = 'STAGE' # 
stn_name = ['MHO','DGL','OLD']
stn_thr = [2.5,2.3,2.3]

sc1_wl = load_dss(hydro1,stn_name, C_part)
sc2_wl  = load_dss(hydro2,stn_name, C_part)
sc1_wl = sc1_wl[stime:etime]
sc2_wl = sc2_wl[stime:etime]

# Load Gate OP
C_part = 'ELEV' # 
stn_list =['MID_GATEOP','GLC_GATEOP','OLD_GATEOP']
#sc1_flow= load_dss(SDG1,flow_list, C_part)
sc2_gateop = load_dss(SDG2,stn_list, C_part)
sc2_gateop = sc2_gateop[stime:etime]
sc2_gateop.rename(columns={'MID_GATEOP':'MHO','GLC_GATEOP':'DGL','OLD_GATEOP':'OLD'}, inplace=True) # renaming to make it easier later.



.\studies\FPV2Ma\output\FPV2Ma_SDG.dss STAGE
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 280512 entries, 2016-01-01 00:00:00 to 2023-12-31 23:45:00
Freq: 15min
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   GLC_GATE_DOWN  280512 non-null  float64
 1   GLC_GATE_UP    280512 non-null  float64
 2   MID_GATE_DOWN  280512 non-null  float64
 3   MID_GATE_UP    280512 non-null  float64
 4   OLD_GATE_DOWN  280512 non-null  float64
 5   OLD_GATE_UP    280512 non-null  float64
dtypes: float64(6)
memory usage: 15.0 MB
.\studies\FPV2Ma\output\FPV2Ma_SDG.dss DEVICE-FLOW
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 280512 entries, 2016-01-01 00:00:00 to 2023-12-31 23:45:00
Freq: 15min
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   GLC_FLOW_FISH  280512 non-null  float64
 1   MID_FLOW_FISH  280512 non-null  float64
 2   MID_FL

In [6]:
thr_vel = 8
mask = (sc2_flow.index.month>=5) & (sc2_flow.index.month<12) # time when gate are in operation

# Make velocity df

def calc_vel(flow,stage_up, bottom_elev, width):
    #velocity is flow/cross-section
    xs = (stage_up-bottom_elev)*width
    vel = flow/xs
    return vel

vel=pd.DataFrame()
vel['GLC'] = calc_vel(sc2_flow.GLC_FLOW_FISH,sc2_stage.GLC_GATE_UP,gatef['bottom_elev'][0], gatef['width'][0] )
vel['MID'] = calc_vel(sc2_flow.MID_FLOW_FISH,sc2_stage.MID_GATE_UP,gatef['bottom_elev'][1], gatef['width'][1] )
vel['OLD'] = calc_vel(sc2_flow.OLD_FLOW_FISH,sc2_stage.OLD_GATE_UP,gatef['bottom_elev'][2], gatef['width'][2] )



In [ ]:
j = 0
station = "DGL"
gate_loc = gatef['ID'][j]
name = gatef['name'][j]
start_zoom = '%s-07-21'%year
end_zoom = '%s-07-28'%year
gate_up = (sc2_gateop.iloc[:,j] >= 10)
vel_zoom = vel[gate_loc][start_zoom:end_zoom]
# print(vel_zoom)
vel_zoom_df = pd.DataFrame(vel_zoom)
vel_zoom_df['datetime'] = vel_zoom_df.index
vel_zoom_df = vel_zoom_df.reset_index(drop=True)
#To count proportion
vel_zoom_df['Velocity_Category'] = np.where(vel_zoom_df['GLC'] >= 8, "Over 8ft/s", "Under 8ft/s")
#.shift shift value down and compare each value with the previous row; increase value when rows are different
vel_zoom_df['consecutive_groups'] = (vel_zoom_df['Velocity_Category'] != vel_zoom_df['Velocity_Category'].shift()).cumsum()
# print(vel_zoom_df)
vel_zoom_df['min_datetime'] = vel_zoom_df.groupby('consecutive_groups')['datetime'].transform('min')
vel_zoom_df['max_datetime'] = vel_zoom_df.groupby('consecutive_groups')['datetime'].transform('max')
vel_zoom_df['date'] = vel_zoom_df['datetime'].dt.date.astype(str)
consecutive_streaks_vel = vel_zoom_df.groupby(['consecutive_groups', 'Velocity_Category', 'min_datetime', 'max_datetime']).size().reset_index(name='count')
consecutive_streaks_vel['streak_duration'] = consecutive_streaks_vel['count'] * 15 / 60
consecutive_streaks_vel_clean = consecutive_streaks_vel.drop(['consecutive_groups', 'Velocity_Category', 'max_datetime'], axis=1)
merged_df = pd.merge(vel_zoom_df, consecutive_streaks_vel_clean,left_on="min_datetime", right_on="min_datetime")
print(merged_df.head(5))
over_streaks = consecutive_streaks_vel[consecutive_streaks_vel['Velocity_Category'] == 'Over 8ft/s']
under_streaks = consecutive_streaks_vel[consecutive_streaks_vel['Velocity_Category'] == 'Under 8ft/s']

# Total duration for over and under 8ft/s
total_over_duration = over_streaks['count'].sum()
total_under_duration = under_streaks['count'].sum()

over_streak_duration =  over_streaks['count'] * 15 / 60
under_streak_duration = under_streaks['count'] * 15 / 60

# Group by the Velocity Category and count the number of days in each category
vel_count_df = vel_zoom_df.groupby('Velocity_Category').size().reset_index(name='count')

total_time_df = pd.DataFrame({
    'Velocity_Category': ['Over 8ft/s', 'Under 8ft/s'],
    'Total Hours': [total_over_duration * 15 / 60, total_under_duration * 15 / 60]
})

consecutive_streaks_vel.rename(columns={
    "Velocity_Category": "Status"},
    inplace = True)
# print(consecutive_streaks_vel)

        GLC            datetime Velocity_Category  consecutive_groups  \
0 -0.585621 2021-07-21 00:00:00       Under 8ft/s                   1   
1 -0.630378 2021-07-21 00:15:00       Under 8ft/s                   1   
2 -0.673443 2021-07-21 00:30:00       Under 8ft/s                   1   
3 -0.706418 2021-07-21 00:45:00       Under 8ft/s                   1   
4 -0.728369 2021-07-21 01:00:00       Under 8ft/s                   1   

  min_datetime        max_datetime        date  count  streak_duration  
0   2021-07-21 2021-07-21 05:00:00  2021-07-21     21             5.25  
1   2021-07-21 2021-07-21 05:00:00  2021-07-21     21             5.25  
2   2021-07-21 2021-07-21 05:00:00  2021-07-21     21             5.25  
3   2021-07-21 2021-07-21 05:00:00  2021-07-21     21             5.25  
4   2021-07-21 2021-07-21 05:00:00  2021-07-21     21             5.25  


In [ ]:
gate_up = (sc2_gateop.iloc[:,0] >= 10)
gate_up = gate_up[start_zoom:end_zoom]
gate_up_df = pd.DataFrame(gate_up)
gate_up_df['datetime'] = gate_up_df.index
gate_up_df = gate_up_df.reset_index(drop=True)
gate_up_df['consecutive_groups'] = (gate_up_df['DGL'] != gate_up_df['DGL'].shift()).cumsum()
gate_up_df['min_datetime'] = gate_up_df.groupby('consecutive_groups')['datetime'].transform('min')
gate_up_df['max_datetime'] = gate_up_df.groupby('consecutive_groups')['datetime'].transform('max')
consecutive_streaks = gate_up_df.groupby(['consecutive_groups', 'DGL', 'min_datetime', 'max_datetime']).size().reset_index(name='count')
consecutive_streaks['streak_duration'] = consecutive_streaks['count'] * 15 / 60
consecutive_streaks_clean = consecutive_streaks.drop(['DGL', 'consecutive_groups', 'max_datetime'], axis = 1)
merged_gate_df = pd.merge(gate_up_df, consecutive_streaks_clean,left_on="min_datetime", right_on="min_datetime")
# print(merged_gate_df)
merged_gate_df = merged_gate_df.drop(['consecutive_groups'], axis=1)
merged_gate_df = merged_gate_df.rename(columns={"min_datetime": "gate_min_datetime", 
                                                "max_datetime": "gate_max_datetime",
                                                "count": "gate_count",
                                                "streak_duration": "gate_streak_duration"})
# merged_gate_df['date'] = merged_gate_df['datetime'].dt.date.astype(str)
# consecutive_streaks = consecutive_streaks[consecutive_streaks['date'] == target_date]
# consecutive_streaks = consecutive_streaks[consecutive_streaks['date'] == '2021-07-21']
consecutive_streaks.rename(columns={
    "DGL": "Status"},
    inplace = True)


In [ ]:
joint_df = pd.concat([consecutive_streaks, consecutive_streaks_vel])

chart = alt.Chart(joint_df).mark_rect(size=20, opacity=0.7).encode(
    y=alt.Y('Status:O', title='Status', axis=alt.Axis(titleFontSize=14)),
    x=alt.X('min_datetime:T', title='Date Time', axis=alt.Axis(titleFontSize=14)),
    x2='max_datetime:T',
    color="Status:O",
    tooltip=[
        alt.Tooltip('Status:O', title='Status'),
        # alt.Tooltip('min_datetime:T', title='Start Time'),
        # alt.Tooltip('max_datetime:T', title='End Time'),
        alt.Tooltip('streak_duration:Q', title='Duration(hour)')]
).properties(
    title={
        "text": ["Gate And Water Velocity Status Throughout the Day"],
        "subtitle": "Gate Open/Close, Water Velocity Over/Under 8m/s Over Time",
        "fontSize": 16,
        "subtitleFontSize": 12,
        "anchor": "start",
        "color": "black"
    },
    width=600,
    height=400
).configure_axis(
    grid=True,
    labelFontSize=12,
    titleFontSize=14,
).configure_title(
    fontSize=18,
    anchor='start'
)

# Show the chart
chart.show()

alt.Chart(...)

In [ ]:
full_merged_df = pd.merge(merged_df, merged_gate_df, left_on="datetime", right_on="datetime")
full_merged_df['time_unit'] = 0.25
full_merged_df['DGL'] = np.where(full_merged_df['DGL'], "Open", "Closed")
xaxis_values = full_merged_df['date'].unique()
print(full_merged_df.head(5))

     GLC  datetime            Velocity_Category  consecutive_groups min_datetime        max_datetime        date         count  streak_duration DGL     gate_min_datetime   gate_max_datetime    gate_count  gate_streak_duration  time_unit
0   -0.59 2021-07-21 00:00:00  Under 8ft/s        1                 2021-07-21 00:00:00 2021-07-21 05:00:00  2021-07-21  21    5.25              Closed 2021-07-21 00:00:00 2021-07-21 04:00:00  17          4.25                 0.25      
1   -0.63 2021-07-21 00:15:00  Under 8ft/s        1                 2021-07-21 00:00:00 2021-07-21 05:00:00  2021-07-21  21    5.25              Closed 2021-07-21 00:00:00 2021-07-21 04:00:00  17          4.25                 0.25      
2   -0.67 2021-07-21 00:30:00  Under 8ft/s        1                 2021-07-21 00:00:00 2021-07-21 05:00:00  2021-07-21  21    5.25              Closed 2021-07-21 00:00:00 2021-07-21 04:00:00  17          4.25                 0.25      
3   -0.71 2021-07-21 00:45:00  Under 8ft/s        1 

In [14]:
interval = alt.selection_interval(encodings=['x'])
line = alt.Chart(full_merged_df).mark_line(color = "darkblue").encode(
    x=alt.X('yearmonthdatehoursminutes(datetime):T', title='Datetime', axis=alt.Axis(format='%b %d, %Y', 
                                                                                     labelAngle=-45,
                                                                                     title='Date')),
    y=alt.Y('GLC:Q', title='Velocity (ft/s)'),
).properties(
    width=500,
    height=300,
    title='Velocity Through Fish Passage-Zoomed',
    # fontSize = 16
).add_selection(
    interval
)

bar_chart = alt.Chart(full_merged_df).mark_bar().encode(
    x=alt.X('Velocity_Category:N', title='Velocity Category'),
    y=alt.Y('sum(time_unit):Q', title='Total Time (Hours)'),
    color=alt.Color('DGL:N', title='Gate Status', scale=alt.Scale(scheme='dark2')),
    tooltip = alt.Tooltip('sum(time_unit):Q', title="Total Time (Hours)")
).properties(
    width=300,
    height=300,
    title="Comparison of Hours by Velocity Category and Gate Status",
    # fontSize = 16
).transform_filter(
    interval
)

area_dgl_true = alt.Chart(full_merged_df).mark_rect(
    # opacity=0.006,
    color='orange'
).encode(
    x='gate_min_datetime:T',
    x2='gate_max_datetime:T',
    opacity=alt.value(0.008)
).transform_filter(
    alt.datum.DGL == "Open"
)
yrule = (
    alt.Chart().mark_rule(strokeDash=[12, 6], size=2, color = 'red').encode(
        y=alt.datum(8))
).properties(
    width=500,
    height=300
).encode(
        tooltip=alt.TooltipValue('8 ft/s Threshold')  # Static tooltip for the rule line
    )

nearest = alt.selection_point(nearest=True, on="pointerover",
                              fields=["datetime"], empty=False)
points = line.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)
# Draw a rule at the location of the selection
rules = alt.Chart(full_merged_df).transform_calculate(
    FlowVelocityDuration = "'Flow ' + datum.Velocity_Category + ' duration is ' + datum.streak_duration + ' hours'",
    GateStatusDuration = "'Gate ' + datum.DGL + ' duration is ' + datum.gate_streak_duration + ' hours'"
).mark_rule(color="gray").encode(
    x="datetime:T",
    opacity=alt.condition(nearest, alt.value(0.3), alt.value(0)),
    tooltip=[alt.Tooltip('yearmonthdatehoursminutes(datetime):T', title='Datetime'),
             alt.Tooltip('GLC:Q', title= "Velocity (ft/s)", format=".2f"),
             alt.Tooltip('FlowVelocityDuration:N', title="Flow Velocity Duration"),
             alt.Tooltip('GateStatusDuration:N', title="Gate Status Duration")
             ],
).add_params(nearest)

combined_chart = alt.hconcat(
    alt.layer(line, points, yrule, rules, area_dgl_true),
    bar_chart
    ).configure_title(
        fontSize=17,
        color = "black",
        font='Arial'
    )

combined_chart 

C:\Users\Inigo\AppData\Local\Temp\ipykernel_21044\2480676262.py:12: AltairDeprecationWarning: Deprecated in `altair=5.0.0`. Use add_params instead.
  ).add_selection(


alt.HConcatChart(...)

In [17]:
clean_merged_df = full_merged_df.drop(["gate_min_datetime",
                                      "gate_max_datetime",
                                      "consecutive_groups",
                                      "min_datetime",
                                      "max_datetime",
                                      "gate_count",
                                      "date",
                                      "count"],axis=1)
clean_merged_df_renamed = clean_merged_df.rename(columns={
    "GLC": "velocity",
    "DGL": "gate_status",
    "gate_streak_duration": "gate_open_close_duration_hr",
    "streak_duration":"velocity_over_under_8_duration_hr",
    "Velocity_Category": "velocity_over_under_8_status"
})
# print(clean_merged_df_renamed.columns)
clean_merged_df_renamed = clean_merged_df_renamed[["datetime", 
                                                   "velocity", 
                                                   "velocity_over_under_8_status",
                                                   "velocity_over_under_8_duration_hr",
                                                   "gate_status",
                                                   "gate_open_close_duration_hr"]]
print(clean_merged_df_renamed.head(5))

  datetime             velocity velocity_over_under_8_status  velocity_over_under_8_duration_hr gate_status  gate_open_close_duration_hr
0 2021-07-21 00:00:00 -0.59      Under 8ft/s                 5.25                                Closed     4.25                        
1 2021-07-21 00:15:00 -0.63      Under 8ft/s                 5.25                                Closed     4.25                        
2 2021-07-21 00:30:00 -0.67      Under 8ft/s                 5.25                                Closed     4.25                        
3 2021-07-21 00:45:00 -0.71      Under 8ft/s                 5.25                                Closed     4.25                        
4 2021-07-21 01:00:00 -0.73      Under 8ft/s                 5.25                                Closed     4.25                        
